# PSTAT 134 Final Project
## Netflix Recommender System
Mollie Jiang, Sean Reagan, Sharon Lee, Uy Tran

In [9]:
import pandas as pd

recipe_df = pd.read_csv('data/recipes.csv')
print(recipe_df.head())

   recipe_id                        recipe_name  aver_rate  \
0     222388                     Homemade Bacon   5.000000   
1     240488  Pork Loin, Apples, and Sauerkraut   4.764706   
2     218939   Foolproof Rosemary Chicken Wings   4.571429   
3      87211              Chicken Pesto Paninis   4.625000   
4     245714                 Potato Bacon Pizza   4.500000   

                                           image_url  review_nums  \
0  https://images.media-allrecipes.com/userphotos...            3   
1  https://images.media-allrecipes.com/userphotos...           29   
2  https://images.media-allrecipes.com/userphotos...           12   
3  https://images.media-allrecipes.com/userphotos...          163   
4  https://images.media-allrecipes.com/userphotos...            2   

                                         ingredients  \
0  pork belly^smoked paprika^kosher salt^ground b...   
1  sauerkraut drained^Granny Smith apples sliced^...   
2  chicken wings^sprigs rosemary^head garlic

# Data Cleaning

In [14]:
print(len(recipe_df))
print(recipe_df.isnull().sum())

49698
recipe_id             0
recipe_name           0
aver_rate             0
image_url             0
review_nums           0
ingredients           0
cooking_directions    0
nutritions            0
reviews               0
dtype: int64


This dataset has 49698 rows, and no missing values so we do not have to deal with any null values.